# XML to CSV

To receive some data entries to work with in the database, I use [https://export.arxiv.org](https://export.arxiv.org) via the query endpoint.

The query: [https://export.arxiv.org/api/query?search_query=cat:cs.*&start=0&max_results=300&sortBy=submittedDate&sortOrder=descending
](https://export.arxiv.org/api/query?search_query=cat:cs.*&start=0&max_results=300&sortBy=submittedDate&sortOrder=descending
).

In [62]:
import os, ssl
import urllib, urllib.request

# Ignores certificate verification for now
ctx = ssl._create_unverified_context()

url = "https://export.arxiv.org/api/query?search_query=cat:cs.*&start=0&max_results=50&sortBy=submittedDate&sortOrder=descending"
data = urllib.request.urlopen(url, context=ctx)
content = data.read().decode('utf-8')

XML_PATH = 'arxiv_papers.xml'

if os.path.exists(XML_PATH):
    os.remove(XML_PATH)

with open(XML_PATH, 'w') as file:
    file.write(content)

In [63]:
import xml.etree.ElementTree as ET
import csv

XML_PATH = 'arxiv_papers.xml'
CSV_PATH = 'arxiv_papers.csv'

tree = ET.parse(XML_PATH)
root = tree.getroot()

if os.path.exists(CSV_PATH):
    os.remove(CSV_PATH)

with open(CSV_PATH, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    csvwriter.writerow(['title', 'author', 'published'])
    
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        title = entry.find('{http://www.w3.org/2005/Atom}title').text
        author = ', '.join([author.find('{http://www.w3.org/2005/Atom}name').text for author in entry.findall('{http://www.w3.org/2005/Atom}author')])
        published = entry.find('{http://www.w3.org/2005/Atom}published').text
        
        csvwriter.writerow([title, author, published])

In [50]:
%pip install arxiv
import arxiv
import csv

# Set up the search query
search = arxiv.Search(
    max_results = 300,
)

# Check if the search returns results
results = list(search.results())
if not results:
    print("No results found for the search query.")
else:
    print(f"Found {len(results)} results.")


if os.path.exists('arxiv_papers.csv'):
    os.remove('arxiv_papers.csv')

# Prepare CSV file
with open('arxiv_papers.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Authors', 'Published', 'DOI', 'Abstract'])

    # Fetch and write papers
    for paper in search.results():
        authors = ', '.join([author.name for author in paper.authors])
        writer.writerow([
            paper.title,
            authors,
            paper.published.strftime('%Y-%m-%d'),
            paper.doi or 'N/A',
            paper.summary.replace('\n', ' ')
        ])

print("Data fetched and saved to arxiv_papers.csv")

Note: you may need to restart the kernel to use updated packages.


/var/folders/xh/1k7xrdlj14v_t7r8cj2v4q7m0000gn/T/ipykernel_8657/3661450599.py:11: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  results = list(search.results())


No results found for the search query.


/var/folders/xh/1k7xrdlj14v_t7r8cj2v4q7m0000gn/T/ipykernel_8657/3661450599.py:27: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for paper in search.results():


Data fetched and saved to arxiv_papers.csv
